# Question 1

In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/05 17:58:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.version

'3.3.2'

# Question 2

In [4]:
from pyspark.sql import types

In [5]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [6]:
df = spark.read \
    .option('header', 'true') \
    .schema(schema) \
    .csv('data/fhvhv_tripdata_2021-06.csv.gz')

In [7]:
df = df.repartition(12)


In [8]:
df.write.parquet('data/fhvhv/2021/06/')

# Question 3

In [9]:
df.registerTempTable('fhv')

/home/lilygrier/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [14]:
spark.sql('''
SELECT COUNT(*)
FROM fhv
WHERE date_trunc('day', pickup_datetime) = '2021-06-15'
''').show()

+--------+
|count(1)|
+--------+
|  452470|
+--------+



In [15]:
from pyspark.sql import functions as F


# Question 4

In [19]:
df_duration = df.withColumn(
    "trip_duration", 
    (F.col("dropoff_datetime").cast("long") - F.col("pickup_datetime").cast("long"))/3600.)


In [23]:
df_duration.select(F.max('trip_duration')).show()

+------------------+
|max(trip_duration)|
+------------------+
|  66.8788888888889|
+------------------+



# Question 5

Port 4040

# Question 6

In [32]:
df_zones = spark.read \
.option('header', 'true') \
.csv('data/taxi_zone_lookup.csv')

In [37]:
df_zones.registerTempTable('zones')

/home/lilygrier/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [44]:
spark.sql('''
SELECT COUNT(*) as zone_count, Zone
FROM fhv JOIN zones
ON zones.LocationID = fhv.PULocationID
GROUP BY Zone
ORDER BY zone_count DESC
LIMIT 1
''').show()

+----------+-------------------+
|zone_count|               Zone|
+----------+-------------------+
|    231279|Crown Heights North|
+----------+-------------------+

